In [1]:
import sys
sys.path.append("../..")

In [2]:
from kai.model_provider import ModelProvider
from kai.models.kai_config import KaiConfigModels

model_config = KaiConfigModels(
    provider="ChatOpenAI",
    args={
        "model": "gpt-4",
    }
)

model_provider = ModelProvider(model_config)

print(model_config.llama_header)

/home/jonah/Projects/github.com/konveyor-ecosystem/kai-jonah/.venv/lib64/python3.11/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/home/jonah/Projects/github.com/konveyor-ecosystem/kai-jonah/.venv/lib64/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


None


In [2]:
response = model_provider.llm.invoke(
"""
Migrate the following file from Jakarta EE to Quarkus. A similar migration has been detected. Use the following description in your migration.  Make sure to explicitly note any files you delete, modify, or create.

File:
```java
// 
client/src/main/java/org/jboss/as/quickstarts/ejb/remote/client/RemoteEJBClient.java
package org.example.ejb.remote.client;

import org.example.ejb.remote.stateful.RemotePhoneNumberGenerator;
import org.example.ejb.remote.stateless.RemoteNameCapitalizer;

import javax.naming.Context;
import javax.naming.InitialContext;
import javax.naming.NamingException;
import java.util.Hashtable;

public class RemoteEJBClientClass {
    private static final String HTTP = "http";

    public static void main(String[] args) throws Exception {
        // Invoke a stateless bean
        invokeStatelessBean("john doe");

        // Invoke a stateful bean
        invokeStatefulBean();
    }

    /**
     * Looks up a stateless bean and invokes on it
     *
     * @throws NamingException
     */
    private static void invokeStatelessBean(String name) throws NamingException {
        // Lookup the remote stateless name capitalizer
        final RemoteNameCapitalizer statelessRemoteNameCapitalizer = lookupRemoteStatelessNameCapitalizer();
        System.out.println("Obtained a remote stateless name capitalizer for invocation");
        // Invoke on the remote name capitalizer
        String capitalizedName = statelessRemoteNameCapitalizer.capitalizeName(name);
        System.out.println("Capitalized name: " + capitalizedName);
    }

    /**
     * Looks up a stateful bean and invokes on it
     *
     * @throws NamingException
     */
    private static void invokeStatefulBean() throws NamingException {
        // Lookup the remote stateful phone number generator
        final RemotePhoneNumberGenerator statefulRemotePhoneNumberGenerator = lookupRemoteStatefulPhoneNumberGenerator();
        System.out.println("Obtained a remote stateful phone number generator for invocation");
        // Generate and print a random phone number
        String phoneNumber = statefulRemotePhoneNumberGenerator.generatePhoneNumber();
        System.out.println("Generated phone number: " + phoneNumber);
    }

    /**
     * Looks up and returns the proxy to remote stateless name capitalizer bean
     *
     * @return
     * @throws NamingException
     */
    private static RemoteNameCapitalizer lookupRemoteStatelessNameCapitalizer() throws NamingException {
        final Hashtable<String, String> jndiProperties = new Hashtable<>();
        jndiProperties.put(Context.INITIAL_CONTEXT_FACTORY, "org.wildfly.naming.client.WildFlyInitialContextFactory");
        jndiProperties.put(Context.PROVIDER_URL, "http://localhost:8080/wildfly-services");
        final Context context = new InitialContext(jndiProperties);
        return (RemoteNameCapitalizer) context.lookup("ejb:/ejb-remote-server-side/NameCapitalizerBean!"
                + RemoteNameCapitalizer.class.getName());
    }

    /**
     * Looks up and returns the proxy to remote stateful phone number generator bean
     *
     * @return
     * @throws NamingException
     */
    private static RemotePhoneNumberGenerator lookupRemoteStatefulPhoneNumberGenerator() throws NamingException {
        final Hashtable<String, String> jndiProperties = new Hashtable<>();
        jndiProperties.put(Context.INITIAL_CONTEXT_FACTORY, "org.wildfly.naming.client.WildFlyInitialContextFactory");
        jndiProperties.put(Context.PROVIDER_URL, "http://localhost:8080/wildfly-services");
        final Context context = new InitialContext(jndiProperties);
        return (RemotePhoneNumberGenerator) context.lookup("ejb:/ejb-remote-server-side/PhoneNumberGeneratorBean!"
                + RemotePhoneNumberGenerator.class.getName() + "?stateful");
    }
}

```

Similar Migration Description:
To migrate the application from Jakarta EE to Quarkus, several significant changes were required across the files:

1. **RemoteEJBClient.java to RemoteRestClient.java**:
   - **Technology Shift**: The original `RemoteEJBClient.java` used EJBs with JNDI lookups to access remote stateless and stateful beans. In contrast, `RemoteRestClient.java` employs RESTful web services and the MicroProfile Rest Client for remote interactions.
   - **Configuration**: Quarkus uses annotations like `@RestClient` and `@RegisterRestClient(configKey="api-key")` to inject REST clients, simplifying the configuration compared to JNDI properties.
   - **Invocation**: The invocation logic in `RemoteEJBClient.java` involves manual lookups and method calls on EJBs, whereas `RemoteRestClient.java` directly calls methods on injected REST clients (`CounterClient` and `CalculatorClient`).
   - **Logging and Exception Handling**: Quarkus uses `Log` for logging instead of `System.out.println`. Exception handling is also simplified with runtime exceptions in Quarkus.

2. **RemoteCalculator and RemoteCounter to REST Clients**:
   - The `RemoteCalculator` and `RemoteCounter` EJBs in Jakarta EE were replaced with REST clients (`CalculatorClient` and `CounterClient`) in Quarkus.
   - **Interfaces**: The EJB interfaces were replaced with Java interfaces annotated with JAX-RS annotations like `@GET`, `@Path`, and `@QueryParam` to define REST endpoints.
   - **Dependencies**: EJB dependencies and configurations were removed in favor of REST client dependencies and annotations.

### Detailed Comparison:

**RemoteEJBClient.java** (Jakarta EE):
- Uses JNDI lookups to access remote EJBs (`RemoteCalculator` and `RemoteCounter`).
- Example of lookup and invocation:
  ```java
  final RemoteCalculator statelessRemoteCalculator = lookupRemoteStatelessCalculator();
  int sum = statelessRemoteCalculator.add(a, b);
  ```
- JNDI properties setup and context initialization.
- Stateless and stateful bean invocation methods:
  ```java
  private static void invokeStatelessBean() throws NamingException { ... }
  private static void invokeStatefulBean() throws NamingException { ... }
  ```

**RemoteRestClient.java** (Quarkus):
- Uses `@RestClient` to inject REST clients (`CounterClient` and `CalculatorClient`).
- Example of REST client injection and invocation:
  ```java
  @RestClient
  CounterClient counterClient;

  int sum = calculatorClient.add(a, b);
  ```
- Simplified configuration using `@RegisterRestClient(configKey="api-key")`.
- Main class implements `QuarkusApplication` and overrides the `run` method:
  ```java
  @Override
  public int run(String... args) throws Exception { ... }
  ```
- Stateless and stateful service invocation methods:
  ```java
  private void invokeStatelessService() { ... }
  private void invokeStatefulService() { ... }
  ```

**CounterClient.java** and **CalculatorClient.java** (Quarkus REST Clients):
- Annotated with JAX-RS annotations to define REST endpoints.
- Example methods:
  ```java
  @GET
  @Path("/increment")
  public void increment();

  @GET
  @Produces(MediaType.TEXT_PLAIN)
  @Path("/add")
  public Integer add(@QueryParam("a") int a, @QueryParam("b") int b);
  ```

These changes reflect the transition from a Jakarta EE EJB-based architecture to a Quarkus REST-based microservices architecture, leveraging the lightweight and flexible nature of Quarkus for modern application development.
"""
)

print(response)


/home/jonah/Projects/github.com/konveyor-ecosystem/kai-jonah/.venv/lib64/python3.11/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/home/jonah/Projects/github.com/konveyor-ecosystem/kai-jonah/.venv/lib64/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


content='Migrated File:\n\n```java\n// client/src/main/java/org/example/rest/client/RemoteRestClient.java\npackage org.example.rest.client;\n\nimport org.example.rest.stateful.RemotePhoneNumberGenerator;\nimport org.example.rest.stateless.RemoteNameCapitalizer;\nimport org.eclipse.microprofile.rest.client.inject.RestClient;\nimport javax.enterprise.context.control.ActivateRequestContext;\nimport javax.ws.rs.core.MediaType;\nimport javax.ws.rs.GET;\nimport javax.ws.rs.Path;\nimport javax.ws.rs.Produces;\nimport javax.ws.rs.QueryParam;\n\npublic class RemoteRestClientClass {\n\n    @RestClient\n    RemoteNameCapitalizer nameCapitalizer;\n\n    @RestClient\n    RemotePhoneNumberGenerator phoneNumberGenerator;\n\n    public static void main(String[] args) throws Exception {\n        // Invoke a stateless service\n        invokeStatelessService("john doe");\n\n        // Invoke a stateful service\n        invokeStatefulService();\n    }\n\n    /**\n     * Invokes a stateless service\n     *

In [3]:
from jinja2 import Environment, FileSystemLoader, StrictUndefined

jinja_env = Environment(
    loader=FileSystemLoader("templates"),
    undefined=StrictUndefined,
    trim_blocks=True,
    lstrip_blocks=True,
)

test_template = jinja_env.get_template("test.jinja")

jinja_vars = {
    "src_file_name": "test.java",
    "src_file_language": "java",
    "src_file_contents": open("original.java", "r").read(),
    "issues": [
        {
            "analysis_message": "Yo, this stuff is messed up!",
            "analysis_line_number": 0,
            "solved_example_diff": ""
        },
    ],
    "model_provider": model_provider,
}

test_render = test_template.render(jinja_vars)

print(test_render)

False

I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your

In [ ]:
print(model_provider.)